In [1]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import json


In [2]:
spark = SparkSession.builder.appName("Steam Game Analysis").getOrCreate()

In [3]:
with open("./input/steamspy/detailed/steam_spy_detailed.json", "r") as f:
    # with open('/Final/input/steamspy/detailed/steam_spy_detailed.json', 'r') as f:
    raw_file = json.load(f)


# Return format for an app:

- appid - Steam Application ID. If it's 999999, then data for this application is hidden on developer's request, sorry.
- name - game's name
- ~~developer - comma separated list of the developers of the game~~
- ~~publisher - comma separated list of the publishers of the game~~
- ~~score_rank - score rank of the game based on user reviews~~
- ~~owners - owners of this application on Steam as a range.~~
- average_forever - average playtime since March 2009. In minutes.
- ~~average_2weeks - average playtime in the last two weeks. In minutes.~~
- median_forever - median playtime since March 2009. In minutes.
- ~~median_2weeks - median playtime in the last two weeks. In minutes.~~
- ~~ccu - peak CCU yesterday.~~
- ~~price - current US price in cents.~~
- initialprice - original US price in cents.
- ~~discount - current discount in percents.~~
- ~~tags - game's tags with votes in JSON array.~~
- languages - list of supported languages.
- genre - list of genres.

Strikethrough indicates dropped variable not used in our analysis


In [4]:
for key in raw_file.keys():
    del raw_file[key]["developer"]
    del raw_file[key]["publisher"]
    del raw_file[key]["userscore"]
    del raw_file[key]["owners"]
    del raw_file[key]["average_2weeks"]
    del raw_file[key]["median_2weeks"]
    del raw_file[key]["score_rank"]
    del raw_file[key]["price"]
    del raw_file[key]["discount"]
    del raw_file[key]["ccu"]
    del raw_file[key]["tags"]
    del raw_file[key]["appid"]


In [5]:
full_data = pd.DataFrame(
    raw_file["570"],
    index=[
        "570",
    ],
)
for n, key in enumerate(raw_file.keys()):
    if n == 0:
        continue
    df = pd.DataFrame(
        raw_file[key],
        index=[
            key,
        ],
    )
    full_data = pd.concat([full_data, df])
full_data


,name,positive,negative,average_forever,median_forever,initialprice,languages,genre
570,Dota 2,1457539,292073,36947,971,0,"English, Bulgarian, Czech, Danish, Dutch, Finn...","Action, Free to Play, Strategy"
730,Counter-Strike: Global Offensive,5683636,757108,28757,6475,0,"English, Czech, Danish, Dutch, Finnish, French...","Action, Free to Play"
578080,PUBG: BATTLEGROUNDS,1140088,888932,22351,7452,0,"English, Korean, Simplified Chinese, French, G...","Action, Adventure, Free to Play, Massively Mul..."
1063730,New World,153855,73173,6166,2792,3999,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Massively Multiplayer, RPG"
440,Team Fortress 2,813449,56122,11986,326,0,"English, Danish, Dutch, Finnish, French, Germa...","Action, Free to Play"
...,...,...,...,...,...,...,...,...
1406980,Road Tripper,1,1,0,0,0,English,"Adventure, Indie, RPG, Simulation, Strategy, E..."
1209610,Virtually Real Life,2,0,0,0,399,English,"Adventure, Indie"
1503000,The Chaos Gene,3,0,0,0,1299,English,"Action, Casual, Indie, Early Access"
1506770,Crawler,5,7,0,0,399,English,"Action, Adventure, Indie"


In [6]:
spark_df = spark.createDataFrame(full_data)

In [7]:
spark_df.printSchema()
spark_df.show()

root
 |-- name: string (nullable = true)
 |-- positive: long (nullable = true)
 |-- negative: long (nullable = true)
 |-- average_forever: long (nullable = true)
 |-- median_forever: long (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- genre: string (nullable = true)



Py4JJavaError: An error occurred while calling o44.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (10.84.239.197 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(